In [1]:
import os
import sys
import torch
import numpy as np
import pandas as pd
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print(torch.__version__)



2.2.2


In [2]:
import pandas as pd
import json

books_path = "/Users/mayarios/Desktop/deeplearning/project/cleaned_books_data.csv"
new_json_path = "/Users/mayarios/Desktop/deeplearning/project/cleaned_books_data.json"


df = pd.read_csv(books_path)


df.to_json(new_json_path, orient="records", indent=4)

print(f"New JSON file saved at: {new_json_path}")


New JSON file saved at: /Users/mayarios/Desktop/deeplearning/project/cleaned_books_data.json


In [3]:
import json
from transformers import AutoTokenizer

books_path = "/Users/mayarios/Desktop/deeplearning/project/cleaned_books_data.json"


with open(books_path, 'r') as f:
    books = json.load(f)


print("First few entries in the JSON file:")
print(json.dumps(books[:5], indent=4))


first_english_sentence = books[0]['english']
first_spanish_sentence = books[0]['spanish']

print("\nFirst English sentence:", first_english_sentence)
print("First Spanish sentence:", first_spanish_sentence)


tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
output = tokenizer(first_english_sentence, truncation=True, max_length=100)

print("\n------------Original-------------")
print(first_english_sentence)

print("------------Encoded-------------")
print(output['input_ids'])

print("------------Decoded-------------")
print(tokenizer.decode(output['input_ids']))


First few entries in the JSON file:
[
    {
        "id": "en/Austen_Jane-Sense_and_Sensibility.xml.gz\tes/Austen_Jane-Sense_and_Sensibility.xml.gz\ts1\ts1",
        "english": "Source: Project GutenbergAudiobook available here",
        "spanish": "Source: Wikisource & librodot.com",
        "id_english": "en/Austen_Jane-Sense_and_Sensibility.xml.gz",
        "id_spanish": "es/Austen_Jane-Sense_and_Sensibility.xml.gz",
        "segment_id": "s1",
        "alignment_id": "s1"
    },
    {
        "id": "en/Austen_Jane-Sense_and_Sensibility.xml.gz\tes/Austen_Jane-Sense_and_Sensibility.xml.gz\ts2\ts2",
        "english": "Sense and Sensibility",
        "spanish": "SENTIDO Y SENSIBILIDAD",
        "id_english": "en/Austen_Jane-Sense_and_Sensibility.xml.gz",
        "id_spanish": "es/Austen_Jane-Sense_and_Sensibility.xml.gz",
        "segment_id": "s2",
        "alignment_id": "s2"
    },
    {
        "id": "en/Austen_Jane-Sense_and_Sensibility.xml.gz\tes/Austen_Jane-Sense_and_Sensibilit

In [4]:
import json
from transformers import AutoTokenizer
from datasets import Dataset  


news_path = "/Users/mayarios/Desktop/deeplearning/project/cleaned_news_data.json"


with open(news_path, 'r') as f:
    news = json.load(f)

# Display the first few entries
print("First few entries in the JSON file:")
print(json.dumps(news[:5], indent=4))

# Extract the first English and Spanish sentence
first_entry = news[0]
first_english_sentence = first_entry.get('english', 'No English text found')
first_spanish_sentence = first_entry.get('spanish', 'No Spanish text found')

print("\nFirst English sentence:", first_english_sentence)
print("First Spanish sentence:", first_spanish_sentence)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

# Tokenize a single sentence for demonstration
tokenized_output = tokenizer(first_english_sentence, truncation=True, max_length=100)

# Define the tokenization function
def tokenize_function(examples):
    encoding = tokenizer(examples["english"], padding="max_length", truncation=True)
    
    # Tokenize Spanish text separately
    spanish_encoding = tokenizer(examples["spanish"], padding="max_length", truncation=True)["input_ids"]

    # Ensure labels are included or default to zeros
    labels = examples.get("label", [0] * len(examples["english"]))

    return {
        "input_ids": encoding["input_ids"],  
        "attention_mask": encoding["attention_mask"],  
        "spanish": spanish_encoding,  
        "labels": labels  
    }

# Convert the news data into a Hugging Face Dataset for mapping
dataset = Dataset.from_list(news)  # Convert list to Dataset

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Print results
print("\n------------Original-------------")
print(first_english_sentence)

print("------------Encoded-------------")
print(tokenized_output['input_ids'])

print("------------Decoded-------------")
print(tokenizer.decode(tokenized_output['input_ids']))


First few entries in the JSON file:
[
    {
        "english": "`White mould' or `Sclerotinia rot' is a very destructive plant disease which can affect agricultural crops such as oilseed rape, soybean and sunflower.",
        "spanish": "El `moho blanco' es una enfermedad de las plantas muy destructiva que puede afectar a cultivos de semillas oleaginosas como colza, soja y girasoles."
    },
    {
        "english": "A action-recognition system is then activated: it receives the information from the sensors, compares those with the task model corresponding to the steps of preparing a milk and sugar cup of tea.",
        "spanish": "Entonces se activa un sistema de reconocimiento de acciones: este recibe la informaci\u00f3n de los sensores y la compara con el modelo de tareas correspondientes a los pasos necesarios para la preparaci\u00f3n de una taza de t\u00e9 con leche y az\u00facar."
    },
    {
        "english": "A actions cover demonstration projects implementing innovative ener

Map:   0%|          | 0/112995 [00:00<?, ? examples/s]


------------Original-------------
`White mould' or `Sclerotinia rot' is a very destructive plant disease which can affect agricultural crops such as oilseed rape, soybean and sunflower.
------------Encoded-------------
[101, 1036, 2317, 9587, 21285, 1005, 2030, 1036, 8040, 3917, 4140, 23309, 18672, 1005, 2003, 1037, 2200, 15615, 3269, 4295, 2029, 2064, 7461, 4910, 8765, 2107, 2004, 20631, 13089, 9040, 1010, 25176, 4783, 2319, 1998, 3103, 14156, 1012, 102]
------------Decoded-------------
[CLS] ` white mould ' or ` sclerotinia rot ' is a very destructive plant disease which can affect agricultural crops such as oilseed rape, soybean and sunflower. [SEP]


In [5]:
try:
    import transformers
    import datasets
    import evaluate
    import accelerate
except ImportError:
    !pip install transformers datasets evaluate accelerate
    !pip install datasets



In [6]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [7]:
from datasets import Dataset

# Convert list of dictionaries into a Dataset
dataset = Dataset.from_list(books)

# Now you can access column names
print(dataset.column_names)



['id', 'english', 'spanish', 'id_english', 'id_spanish', 'segment_id', 'alignment_id']


In [8]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from datasets import Dataset, DatasetDict


split_dataset = tokenized_books['train'].train_test_split(test_size=0.1)

# Create a new DatasetDict with the train-test split
books = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


train_dataloader = DataLoader(
    books['train'].remove_columns(["english", "spanish"]),  
    shuffle=True,
    batch_size=256,
    collate_fn=data_collator
)



test_dataloader = DataLoader(
    books['test'].remove_columns(["english", "spanish"]), 
    shuffle=False,
    batch_size=256,
    collate_fn=data_collator
)

print("Train DataLoader ready with batch size:", train_dataloader.batch_size)
print("Test DataLoader ready with batch size:", test_dataloader.batch_size)

NameError: name 'tokenized_books' is not defined

In [ ]:
batch=next(iter(train_dataloader))

In [ ]:
print(batch.keys())
print(batch['input_ids'].shape)
print(batch['input_ids'])

In [ ]:
vocab_size=len(tokenizer.vocab)

In [ ]:
import torch.nn as nn
import torch.optim as optim

class SimpleRNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, output_dim,pad_index):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_index)        
        self.rnn = nn.LSTM(embed_dim, embed_dim, batch_first=True,dropout=0.0)
        self.fc = nn.Linear(embed_dim, output_dim)
    
    def forward(self, x):
        embedding=self.embedding(x)
        output, states = self.rnn(embedding)
        output=self.fc(output[:,-1,:]).squeeze()
        return torch.nn.functional.sigmoid(output)



In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())  # Check if 'labels' is in the batch


In [ ]:
import torch
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from torchmetrics.classification import Accuracy, F1Score

n_epochs = 1
test_loss = []
train_loss = []

model = SimpleRNNClassifier(vocab_size=vocab_size, embed_dim=50, output_dim=1, pad_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.BCELoss()

# Metrics initialization
accuracy_metric = Accuracy()
f1_metric = F1Score()

for epoch in range(n_epochs):
    model.train()
    _loss = []
    
    # Training Loop
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        output = model(batch['input_ids'])
        loss = loss_fn(output, batch['labels'].float())
        loss.backward()
        optimizer.step()
        _loss.append(loss.item())
    
    # Save the last 10 training losses
    train_loss.append(np.mean(_loss[-10:]))
    
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        
        # Evaluation Loop
        for test_batch in test_dataloader:
            test_output = model(test_batch['input_ids'])
            test_label = test_batch['labels'].float()
            loss = loss_fn(test_output, test_label)
            test_loss.append(loss.item())
            
            # Store predictions and true labels for metrics
            all_preds.append(test_output)
            all_labels.append(test_label)

        # Concatenate all predictions and labels
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        
        # Calculate accuracy and F1 score
        accuracy = accuracy_metric(all_preds.round(), all_labels)
        f1 = f1_metric(all_preds.round(), all_labels)
        
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {train_loss[-1]:.4f} - Val Loss: {np.mean(test_loss[-len(test_dataloader):]):.4f}")
    print(f"Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")


In [ ]:
books = "/Users/mayarios/Desktop/deeplearning/project/cleaned_books_data.csv"
news = "/Users/mayarios/Desktop/deeplearning/project/cleaned_news_data.csv"

books_df = pd.read_csv(books)
news_df = pd.read_csv(news)

print("First 5 rows of the books CSV file:")
print(books_df.head())

print("\nFirst 5 rows of the news CSV file:")
print(news_df.head())

In [ ]:
import sys
!{sys.executable} -m pip install datasets

In [ ]:
import json
from transformers import AutoTokenizer

# Define the file path
books_path = "/Users/mayarios/Desktop/Deep Learning/potential project/cleaned_books_data.json"

# Load the JSON file
with open(books_path, 'r') as f:
    books = json.load(f)

# Print the first few entries
print("First few entries in the JSON file:")
print(json.dumps(books[:5], indent=4))

# Access specific fields
first_english_sentence = books[0]['english']
first_spanish_sentence = books[0]['spanish']

print("\nFirst English sentence:", first_english_sentence)
print("First Spanish sentence:", first_spanish_sentence)

# Tokenize the first English sentence
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
output = tokenizer(first_english_sentence, truncation=True, max_length=100)

print("\n------------Original-------------")
print(first_english_sentence)

print("------------Encoded-------------")
print(output['input_ids'])

print("------------Decoded-------------")
print(tokenizer.decode(output['input_ids']))

# Save a subset of the data
subset = books[:10]  # Get the first 10 entries
with open("subset_books.json", "w") as f:
    json.dump(subset, f, indent=4)

print("\nSubset saved to 'subset_books.json'.")

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_books = books.map(tokenize_function, batched=True)
